In [18]:

from cmath import inf
#from mlinsights.plotting import pipeline2str
#from mlinsights.plotting import pipeline2dot
import networkx as nx
from pyvis.network import Network
import matplotlib as mpl
import networkx as nx

from pyvis.network import Network

import matplotlib as mpl



#load pickle evaluated pipelines

from os.path import exists
import matplotlib.pyplot as plt
import numpy as np
import dill as pickle
import sys
import pandas as pd
from tpot import TPOTClassifier
import deap
from deap import creator
sys.path.append('/Users/matsumoton/Git/digen')
from digen import Benchmark

import pydot 
from IPython.display import Image, display

import networkx as nx
import random
import math

import networkx.algorithms as na

class TrieNode:
 
    def __init__(self, primitive = 'root'):
        self.primitive = primitive
        self.path = 'root'
        self.traverse_count = 0
        self.total_cv_score = []
        self.generation = []
        self.children = {}
        self.parents = []
        self.depth = 0
        self.max_score = -inf
        self.min_score = inf
        self.diversity_score = 0
 
class PipelineTrie(object):
 
    def __init__(self):
        self.root = TrieNode("")
        self.graph = {}
        self.metrics = {}
        self.diversity_score = 0
        self.number_of_unique_nodes = 0
        self.unique_nodes_per_depth = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        
    def insert(self, pipeline_str,pipeline_data,pset):

        def prim_to_list(prim, args):
            if isinstance(prim, deap.gp.Terminal):
                return None
            return [prim.name] + args
        def remove_none(obj):
            if isinstance(obj, (list, tuple, set)):
                return type(obj)(remove_none(x) for x in obj if x is not None)
            elif isinstance(obj, dict):
                return type(obj)((remove_none(k), remove_none(v))
                for k, v in obj.items() if k is not None and v is not None)
            else:
                return obj

        pipeline = creator.Individual.from_string(pipeline_str, pset)

        #convert pipeline into a list and change all hyperparameters to None
        tree = []
        stack = []
        for node in pipeline:
            stack.append((node, []))
            while len(stack[-1][1]) == stack[-1][0].arity:
                prim, args = stack.pop()
                tree = prim_to_list(prim, args)
                if len(stack) == 0:
                    break  # If stack is empty, all nodes should have been seen
                
                stack[-1][1].append(tree)
        
        #remove all Nones
        tree = remove_none(tree)
        
        #dfs through the tree and integrate into trie
        stack = []
        stack.append(tree)
        trie_stack = [self.root]

        while stack:
            s = stack.pop()
            node = trie_stack.pop()
            cur_depth = node.depth+1
            
            if (s[0]) not in node.children:
                node.children[(s[0])] = TrieNode(s[0])
                node.children[(s[0])].parents = np.append(node.parents,node)
                #add a value to the root diversity metric
                self.diversity_score =  self.diversity_score + 1/cur_depth
                
                self.number_of_unique_nodes = self.number_of_unique_nodes + 1
                #only keep max 15 depth
                if cur_depth <= 15:
                    self.unique_nodes_per_depth[cur_depth] = self.unique_nodes_per_depth[cur_depth] + 1

            node.children[(s[0])].traverse_count = node.children[(s[0])].traverse_count + 1
            node.children[(s[0])].total_cv_score.append(pipeline_data["internal_cv_score"])
            node.children[(s[0])].generation.append(pipeline_data["generation"])
            node.children[(s[0])].depth = cur_depth
            if not math.isnan(pipeline_data["internal_cv_score"]) and not math.isinf(pipeline_data["internal_cv_score"]):
                node.children[(s[0])].min_score = min(node.children[(s[0])].min_score,pipeline_data["internal_cv_score"])
                node.children[(s[0])].max_score = max(node.children[(s[0])].max_score,pipeline_data["internal_cv_score"])
                self.root.min_score = min(self.root.min_score,pipeline_data["internal_cv_score"])
                self.root.max_score = max(self.root.max_score,pipeline_data["internal_cv_score"])
            if node.path != 'root':
                node.children[(s[0])].path = node.path + '-' + s[0]
            else:
                node.children[(s[0])].path = s[0]
            if len(s[1:]) > 0:
                stack.extend(s[1:])
                for i in range(len(s[1:])):
                    trie_stack.append(node.children[(s[0])])
                    
    def get_networkx_graph(self,depth=100):
        import networkx as nx
        from pyvis.network import Network
        import matplotlib as mpl

        def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
            c1=np.array(mpl.colors.to_rgb(c1))
            c2=np.array(mpl.colors.to_rgb(c2))
            return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

        c1='red' #blue
        c2='green' #green

        graph = pydot.Dot(graph_type='graph') 
        stack = [self.root]
        parent_stack = []

        max_height = depth
        while stack:
            s = stack.pop()
            if s.depth >= max_height:
                continue
            for k in s.children.keys():
                stack.append(s.children[k])
                # temp =  [v for v in s.total_cv_score if not math.isnan(v) and not math.isinf(v)]
                # if len(temp) :
                #     parentnodeaccuracy =(sum(temp)/len(temp))
                #     if parentnodeaccuracy > self.root.max_score:
                #         parentnodeaccuracy = self.root.max_score
                #     parentnodecolor = colorFader(c1,c2,(parentnodeaccuracy-self.root.min_score)/(self.root.max_score-self.root.min_score))
                # else:
                #     parentnodeaccuracy = 'NA'
                #     parentnodecolor = "#666666"
                    
                # temp =  [v for v in s.children[k].total_cv_score if not math.isnan(v) and not math.isinf(v)]
                # if len(temp) :
                #     childaccuracy = (sum(temp)/len(temp))
                #     #floating point 0.00...01 issue
                #     if childaccuracy > self.root.max_score:
                #         childaccuracy = self.root.max_score
                #     childcolor = colorFader(c1,c2,(childaccuracy-self.root.min_score)/(self.root.max_score-self.root.min_score))
                    
                # else:
                #     childaccuracy = 'NA'
                #     childcolor = "#666666"
                if s.path == 'root':
                    graph.add_node(pydot.Node(s.path,label=s.primitive,size=20,color="#ff4f4c"))
                else:
                    graph.add_node(pydot.Node(s.path,label=s.primitive,size=20))
                graph.add_node(pydot.Node(s.children[k].path,label=s.children[k].primitive,size=20))
                
                edge = pydot.Edge(s.path, s.children[k].path,weight=1,color='#515ba3',value=3)
                graph.add_edge(edge)
        self.graph = nx.nx_pydot.from_pydot(graph)
        
    def update_metrics(self,generation):
        self.metrics[generation] = {}


        A = nx.adjacency_matrix(pipeline_trie.graph)
        A=A.todense()
        
        #of leaf nodes
        leafnode_counter = 0
        for i in range(A.shape[0]):
            if A[i].sum() == 1:
                leafnode_counter = leafnode_counter + 1
        self.metrics[generation]["unique_nodes"] = self.number_of_unique_nodes
        self.metrics[generation]["global_efficiency"] = self.diversity_score / (self.number_of_unique_nodes )
        self.metrics[generation]["exploratory_metric"] = self.diversity_score
        self.metrics[generation]["leafnode_counter"] = leafnode_counter
        self.metrics[generation]["unique_nodes_by_depth"] = self.unique_nodes_per_depth
        #self.metrics[generation]["global_efficiency"] = na.global_efficiency(pipeline_trie.graph)
        #self.metrics[generation]["local_efficiency"] = na.local_efficiency(pipeline_trie.graph)
        print(self.metrics[generation])
        

    def display(self,filename):
        import networkx as nx
        from pyvis.network import Network
        import matplotlib as mpl
        nt = Network(height='800px', width='1280px', bgcolor='#ffffff', font_color='black')
        nt.from_nx(self.graph)
        nt.show(filename+'.html')
        nx.write_edgelist(self.graph, filename+".edgelist")
 


 
    

from sklearn.utils import shuffle
from os import makedirs


def extract_labels(df, labelname):
    y = df[labelname].copy(deep=True)
    x = df.drop(labelname, axis=1)
    x, y = shuffle(x, y)
    x = x.to_numpy()
    y = y.to_numpy()
    return x, y



In [4]:
classifier_config_dict = {

    # Classifiers
    'sklearn.naive_bayes.GaussianNB': {
    },

    'sklearn.naive_bayes.BernoulliNB': {
    },

    'sklearn.naive_bayes.MultinomialNB': {
    },

    'sklearn.tree.DecisionTreeClassifier': {
    },

    'sklearn.ensemble.ExtraTreesClassifier': {
    },

    'sklearn.ensemble.RandomForestClassifier': {
    },

    'sklearn.ensemble.GradientBoostingClassifier': {
    },

    'sklearn.neighbors.KNeighborsClassifier': {
    },



    'sklearn.linear_model.LogisticRegression': {
    },

    'sklearn.preprocessing.Binarizer': {
        'threshold': np.arange(0.0, 1.01, 0.05)
    },




    # Preprocesssors

    'sklearn.decomposition.PCA': {
        'iterated_power': range(1, 11)
    },

        'sklearn.kernel_approximation.RBFSampler': {
        'gamma': np.arange(0.0, 1.01, 0.05)
    },

        'sklearn.preprocessing.Normalizer': {
        'norm': ['l1', 'l2', 'max']
    },

}

In [7]:
from distutils.command.config import config
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import sklearn
from deap import creator


n = 100
X = np.random.rand(n,10)
y = np.random.randint(0,2,n)
pipeline_optimizer = TPOTClassifier(generations=1, population_size=10, cv=10, crossover_rate=0.5, mutation_rate=.5,
                                     verbosity=2, n_jobs=32, config_dict = classifier_config_dict)

pipeline_optimizer.fit(X,y)

                                                                            
Generation 1 - Current best internal CV score: 0.61
                                                                            
Best pipeline: GradientBoostingClassifier(Binarizer(input_matrix, threshold=0.75))


TPOTClassifier(config_dict={'sklearn.decomposition.PCA': {'iterated_power': range(1, 11)},
                            'sklearn.ensemble.ExtraTreesClassifier': {},
                            'sklearn.ensemble.GradientBoostingClassifier': {},
                            'sklearn.ensemble.RandomForestClassifier': {},
                            'sklearn.kernel_approximation.RBFSampler': {'gamma': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0....
                            'sklearn.preprocessing.Binarizer': {'threshold': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])},
                            'sklearn.preprocessing.Normalizer': {'norm': ['l1',
                                                                          'l2',
                                                                          'max']},
                            'sklearn.tree.DecisionTreeClassifier': {}},
               crossover_rate=0.5, cv=10, generations=1, mutation_rate=0.5,
               n_jobs=32, population_size=10, verbosity=2)

In [5]:
pipeline_optimizer.evaluated_individuals_.keys()

dict_keys(['ExtraTreesClassifier(input_matrix)', 'RandomForestClassifier(BernoulliNB(input_matrix))', 'LogisticRegression(input_matrix)', 'DecisionTreeClassifier(DecisionTreeClassifier(input_matrix))', 'GradientBoostingClassifier(ExtraTreesClassifier(input_matrix))', 'LogisticRegression(RBFSampler(input_matrix, RBFSampler__gamma=0.25))', 'BernoulliNB(RandomForestClassifier(input_matrix))', 'MultinomialNB(DecisionTreeClassifier(input_matrix))', 'ExtraTreesClassifier(PCA(input_matrix, PCA__iterated_power=5))', 'BernoulliNB(RandomForestClassifier(ExtraTreesClassifier(input_matrix)))', 'ExtraTreesClassifier(ExtraTreesClassifier(input_matrix))', 'BernoulliNB(input_matrix)', 'GradientBoostingClassifier(input_matrix)', 'BernoulliNB(RandomForestClassifier(BernoulliNB(input_matrix)))', 'BernoulliNB(GaussianNB(input_matrix))', 'RandomForestClassifier(RandomForestClassifier(input_matrix))', 'DecisionTreeClassifier(input_matrix)'])

In [5]:
def get_pipeline(pipeline):
    test = creator.Individual.from_string(pipeline, pipeline_optimizer._pset)
    print(test)
    pipeline_fitted = pipeline_optimizer._toolbox.compile(expr=test)
    return pipeline_fitted

In [8]:
pipeline1 = "LogisticRegression(CombineDFs(PCA(input_matrix, PCA__iterated_power=2), PCA(input_matrix, PCA__iterated_power=3)))"
get_pipeline(pipeline1)

LogisticRegression(CombineDFs(PCA(input_matrix, PCA__iterated_power=2), PCA(input_matrix, PCA__iterated_power=3)))


Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('pca-1',
                                                 PCA(iterated_power=2)),
                                                ('pca-2',
                                                 PCA(iterated_power=3))])),
                ('logisticregression', LogisticRegression())])

1 : PCA(input_matrix, PCA__iterated_power=2)

2 : Normalizer(input_matrix, Normalizer__norm=l2)

3 : RBFSampler(input_matrix, RBFSampler__gamma=0.0)

4 : Binarizer(input_matrix, Binarizer__threshold=0.0)

In [10]:
pipeline0 = "LogisticRegression(input_matrix)"
pipeline1 = "LogisticRegression(PCA(input_matrix, PCA__iterated_power=1))"
pipeline2 = "DecisionTreeClassifier(input_matrix)"
pipeline3 = "LogisticRegression(Normalizer(input_matrix, Normalizer__norm=l1))"

In [20]:

pipeline_trie = PipelineTrie()
pipeline_trie.insert(pipeline0, {"internal_cv_score": 1, "generation":1}, pipeline_optimizer._pset)
pipeline_trie.get_networkx_graph(100)
pipeline_trie.update_metrics(4)
pipeline_trie.get_networkx_graph(100)
pipeline_trie.display("test1")

pipeline_trie.insert(pipeline1, {"internal_cv_score": 1, "generation":2}, pipeline_optimizer._pset)
pipeline_trie.get_networkx_graph(100)
pipeline_trie.update_metrics(4)
pipeline_trie.get_networkx_graph(100)
pipeline_trie.display("test2")

pipeline_trie.insert(pipeline2, {"internal_cv_score": 1, "generation":3}, pipeline_optimizer._pset)
pipeline_trie.get_networkx_graph(100)
pipeline_trie.update_metrics(4)
pipeline_trie.get_networkx_graph(100)
pipeline_trie.display("test3")

pipeline_trie.insert(pipeline3, {"internal_cv_score": 1, "generation":1}, pipeline_optimizer._pset)
pipeline_trie.get_networkx_graph(100)
pipeline_trie.update_metrics(4)
# pipeline_trie.insert(pipeline3, {"internal_cv_score": 1, "generation":1}, pipeline_optimizer._pset)
# pipeline_trie.insert(pipeline4, {"internal_cv_score": 1, "generation":1}, pipeline_optimizer._pset)

pipeline_trie.get_networkx_graph(100)
#pipeline_trie.update_metrics(1)
pipeline_trie.display("test4")
# pipeline_trie.metrics

/var/folders/t2/2rb77zbx4fvdt47gjll69t9h0000gt/T/ipykernel_52373/860686335.py:194: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(pipeline_trie.graph)


{'unique_nodes': 1, 'global_efficiency': 1.0, 'exploratory_metric': 1.0, 'leafnode_counter': 2, 'unique_nodes_by_depth': [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'unique_nodes': 2, 'global_efficiency': 0.75, 'exploratory_metric': 1.5, 'leafnode_counter': 2, 'unique_nodes_by_depth': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'unique_nodes': 3, 'global_efficiency': 0.8333333333333334, 'exploratory_metric': 2.5, 'leafnode_counter': 2, 'unique_nodes_by_depth': [1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'unique_nodes': 4, 'global_efficiency': 0.75, 'exploratory_metric': 3.0, 'leafnode_counter': 3, 'unique_nodes_by_depth': [1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
